## <p style="text-align:center">Bond Key Rate Durations calculation in FinancePy</p>


In [1]:

from financepy.utils.date import Date
from financepy.utils.day_count import DayCountTypes
from financepy.utils.frequency import FrequencyTypes
from financepy.products.bonds import *
from financepy.market.curves import *

####################################################################
# FINANCEPY BETA Version 0.270 - This build:  10 Apr 2023 at 20:31 #
#      This software is distributed FREE & WITHOUT ANY WARRANTY    #
#  Report bugs as issues at https://github.com/domokane/FinancePy  #
####################################################################



Modified duration measures a bond's overall price sensitivity to interest rate changes, while key rate durations measure sensitivity at specific points on the yield curve. 

In [2]:
# First, let's create a bond object.


accrual_type, frequencyType, settlementDays, exDiv, calendar = get_bond_market_conventions(
    BondMarkets.UNITED_STATES)

# interest accrues on this date. Original issue date is 01/08/2022
issue_date = Date(31, 7, 2022)
maturity_date = Date(31, 7, 2027)
cpn = 2.75/100
bond = Bond(issue_date, maturity_date, cpn, frequencyType, accrual_type)
settlement_date = Date(20, 3, 2023)  # next settle date for this bond
bond


OBJECT TYPE: Bond
ISSUE DATE: 31-JUL-2022
MATURITY DATE: 31-JUL-2027
COUPON (%): 2.75
FREQUENCY: FrequencyTypes.SEMI_ANNUAL
ACCRUAL TYPE: DayCountTypes.ACT_ACT_ICMA
FACE AMOUNT: 100.0

Below is a screenshot from Bloomberg for this bonds key rate durations. For reference, ISIN is US91282CFB28, and Cusip is 91282CFB2
Please note that, Key rate durations analytics on Bloomberg standard terminal are available only for last data. So, if you want to replicate, you can view it's last data and change the settlement_date above.

![Alt text](data/bg_key_rate_durations.png)

Data taken on 17 March 2023


Key rate durations method of the Bond class takes two mandatory parameters: settlement_date (or valuation date) and Yield to maturity. Optionally, we can provide our own key rate tenors (in years) and our own shift value. 
Shift value defaults to 0.0001 and is used to shift the key rate of a corresponding tenor to assess the change in bond price.
Default key rate tenors are 0.25, 0.5, 1, 2, 3, 4, 5, 7, 10, 20, 30

In [3]:
# the method returns a tuple of key_rate_tenors, key_rate_durations

# US Government Equivalent yield on Bloomberg as of 17 March 2023
ytm = 3.803140/100

# settlement date is the date on which the bond is traded or valued

key_rate_tenors, key_rate_durations = bond.key_rate_durations(settlement_date, ytm)

In [4]:
for tenor, krd in zip(key_rate_tenors, key_rate_durations):
    print(tenor, ">>>", round(krd,6))   

0.25 >>> 0.002773
0.5 >>> 0.005383
1.0 >>> 0.023423
2.0 >>> 0.051905
3.0 >>> 0.075023
4.0 >>> 2.502754
5.0 >>> 1.379869
7.0 >>> 0.0
10.0 >>> 0.0
20.0 >>> 0.0
30.0 >>> 0.0


In [5]:
# Modified duration of a bond and sum of the key rate duration should be close to each other.

mod_dur = bond.modified_duration(settlement_date, ytm)
sum_of_krds = sum(key_rate_durations)

print("Modified duration ", mod_dur)
print("Sum of key rate durations ", sum_of_krds)

Modified duration  4.047346488726142
Sum of key rate durations  4.041129255598644


<p style="text-align:right">prepared by github.com/sagayev </p>